In [1]:
from langchain_groq import ChatGroq

In [2]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='<your_key>', 
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [4]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-50293?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(page_data)

Apply for Senior Data Engineer

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsData & AnalyticsSenior Data EngineerBoston, MassachusettsBecome part of the Converse Team
 
Converse is a 

In [6]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

```json
{
  "role": "Senior Data Engineer",
  "experience": "2 years",
  "skills": [
    "Cloud Computing & Engineering (AWS)",
    "Database Engineering [SQL] (Snowflake)",
    "Programming & Scripting (Python, Shell)",
    "Data Warehousing & ETL",
    "Data Architecture",
    "Data Modelling",
    "Data Analysis",
    "Data visualization (Tableau)",
    "Marketing Technology Systems & Application",
    "API Engineering & Management"
  ],
  "description": "Design and build reusable components, frameworks, and libraries at scale to support analytics products. Identify and solve issues concerning data management to improve data quality, and clean, prepare and optimize data for ingestion and consumption. Build continuous integration, test-driven development and production deployment frameworks. Work with Data Engineering teammates to review, design, code, and test plans to increase knowledge and application of key methodologies. Collaborate with business and Technology partners to build

In [7]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Senior Data Engineer',
 'experience': '2 years',
 'skills': ['Cloud Computing & Engineering (AWS)',
  'Database Engineering [SQL] (Snowflake)',
  'Programming & Scripting (Python, Shell)',
  'Data Warehousing & ETL',
  'Data Architecture',
  'Data Modelling',
  'Data Analysis',
  'Data visualization (Tableau)',
  'Marketing Technology Systems & Application',
  'API Engineering & Management'],
 'description': 'Design and build reusable components, frameworks, and libraries at scale to support analytics products. Identify and solve issues concerning data management to improve data quality, and clean, prepare and optimize data for ingestion and consumption. Build continuous integration, test-driven development and production deployment frameworks. Work with Data Engineering teammates to review, design, code, and test plans to increase knowledge and application of key methodologies. Collaborate with business and Technology partners to build well-defined components that support th

In [8]:
type(json_res)

dict

In [9]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [10]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [12]:
links = collection.query(query_texts=['Experience in python', "Expertiese in React Native"], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-native-portfolio'},
  {'links': 'https://example.com/react-portfolio'}]]

In [13]:
job = json_res
job["skills"]

['Cloud Computing & Engineering (AWS)',
 'Database Engineering [SQL] (Snowflake)',
 'Programming & Scripting (Python, Shell)',
 'Data Warehousing & ETL',
 'Data Architecture',
 'Data Modelling',
 'Data Analysis',
 'Data visualization (Tableau)',
 'Marketing Technology Systems & Application',
 'API Engineering & Management']

In [14]:
links = collection.query(query_texts=job["skills"], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/python-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ios-portfolio'},
  {'links': 'https://example.com/flutter-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/ios-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://exa

In [15]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Dulara, a data engineer at XYZ. XYZ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of XYZ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase XYZ's portfolio: {link_list}
        Remember you are Dulara, DE at XYZ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Data Engineering Solutions for Seamless Business Integration

Dear Hiring Manager,

I came across the Senior Data Engineer role at your organization and was impressed by the job description. As a Data Engineer at XYZ, an AI & Software Consulting company, I believe we can fulfill your needs and empower your enterprise with tailored data engineering solutions.

Our team at XYZ has extensive experience in designing and building reusable components, frameworks, and libraries at scale to support analytics products. We specialize in Cloud Computing & Engineering (AWS), Database Engineering (Snowflake), Programming & Scripting (Python, Shell), Data Warehousing & ETL, Data Architecture, Data Modelling, Data Analysis, and Data Visualization (Tableau). Our expertise in Marketing Technology Systems & Application and API Engineering & Management can help you build well-defined components that support the implementation of product features.

We have a proven track record of deliveri